# Exempel notebook

* Läsa in data från SAS dataset
* Hur man skapar en ny variabel baserat på befintlig variabel
* Hur man konverterar ett string objekt till Pandas datetime variabel
* Några enkla metoder för att förstå sitt data
* Gruppering av data
* Filtrering - välja rader på kriterium
* Flödeslogik och villkor i en funktion applicerad mot dataframe
* Joina dataframes
* Konkatinera dataframes
* Visualisering - Matplotlib
* Objekthantering när dataframes skapas, vy eller fysisikt data - SettingWithWarningCopy
* Plocka rader och kolumner i dataframe med iloc metoden


__En mycket bra resurs att söka hjälp i är pandas egen dokumentation: https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sas.html__

__I Table of Contents finns "Comparison with SAS". I denna så jämför man pandas med SAS utifrån flera olika perspektiv. Jag brukar alltid ha den öppen när jag arbetar med analys i pandas__



In [ ]:
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt

In [ ]:
?pd.read_sas

In [ ]:
type(pd.read_sas)

In [ ]:
type(plt)

In [ ]:
type(pd)

## 1. Läsa in data från SAS format

In [ ]:
df_staff = pd.read_sas('staff.sas7bdat', encoding = 'Latin8')

In [ ]:
# Information om det inlästa datat
df_staff.info()

In [ ]:
# Tittar på de 5 första observationerna
df_staff.head()

### Dataframen innehåller två datatyper: float64 samt object
* float64 - numerisk variabel med decimaler
* object - pandas datatyp för character
* datetime64 - pandas datatyp för datum

## 2. Hur skapar man en ny variabel i Pandas

In [ ]:
df_staff['Double'] = df_staff['Salary'] * 2

In [ ]:
df_staff.head()

## 3. Konvertera till datetimeformat

### Vi behöver konvertera datumvariablerna (Start_Date samt End:Date) till datetime - datumvariabel i Pandas. Nu ligger den som objekt och då kan vi inte applicera datum metoder på denna. För att göra den konverteringen använder vi pandas funktion to_datetime


In [ ]:
?pd.to_datetime

In [ ]:
df_staff['P_start_date'] = pd.to_datetime(df_staff['start_date'])

In [ ]:
df_staff.info()

In [ ]:
df_staff.head()

### Nu kan vi applicera metoder mot datumvariabeln

In [ ]:
type(df_staff['P_start_date'])

In [ ]:
# För att se vilka attribut som finns tillgängliga på en serie
test = df_staff['P_start_date']

In [ ]:
dir(test)

In [ ]:
?test.dt

### Vi skapar år, månad och dag

In [ ]:
df_staff['Year'] = df_staff['P_start_date'].dt.year
df_staff['Month'] = df_staff['P_start_date'].dt.month
df_staff['day'] = df_staff['P_start_date'].dt.month


In [ ]:
df_staff.head()

## 4. Exempel på metoder för att enkelt förstå sitt data

In [ ]:
# Fördelning män och kvinnor
df_staff['Gender'].value_counts()

In [ ]:
# Medelvärde lön
df_staff['Salary'].mean()


In [ ]:
# Univariat statistik på alla numeriska variabler i dataframen
df_staff.describe()

## 5. Gruppering av data. 
### För att gruppera sitt data skapar man ett sk grouped objekt. Ett grouped objekt innehåller metadata som beskriver hur grupperingen är gjord. På detta objekt kan man sedan välja kolumner att ta fram olika statistik på. Detta sker - det vet ni nu - självklart med en lämplig metod.

In [ ]:
# Vi skapar ett grouped objekt på Gender

grouped = df_staff.groupby('Gender')

In [ ]:
type(grouped)

In [ ]:
# Finns det någon skillnad i lön mellan män och kvinnor?. Väljer Serien Salery från groupedby objektet och applicerar 
# metoden mean

grouped['Salary'].mean()

In [ ]:
df_staff.info()

In [ ]:
### Man kan självklart gruppera på mer än en variabel. Observera att man då lägger grupperingsvariablerna i en lista

grouped_multi =  df_staff.groupby(['Year','Gender'])

In [ ]:
round(grouped_multi['Salary'].mean())

## 6. Filtrering - välja rader på kriterium

In [ ]:
# Vi skapar en ny dataframe som enbart innehåller kvinnor
df_staff_female = df_staff[df_staff['Gender'] == 'F']

In [ ]:
df_staff_female.count()

### Vad är det vi gör här egentligen?

In [ ]:
df_staff['Truth'] = df_staff['Gender'] == 'F'

In [ ]:
# Väljer ett subset av variabler
df_staff[['Gender','Truth']].head()

In [ ]:
### Subsetting with Calculated Values - exempel SQL

'''
proc sql;
select Gender, Salary, Bonus,
       Salary * .10 as Bonus
   from orion.employee_payroll
   where calculated Bonus < 3000;
quit; 

'''


In [ ]:
df_staff['Bonus'] = df_staff['Salary'] * 0.1
df_staff_subset = df_staff[df_staff['Bonus'] > 3000][[ 'Gender', 'Salary','Bonus']]

In [ ]:
df_staff_subset.head(10)

In [ ]:
df_staff_subset['Gender'].value_counts()

## 7. Flödeslogik och villkor i en funktion applicerad mot dataframe
### Förfina logiken med egendiefinerad funktion som ancänds av apply metoden i en lambda funktion. Om det är en kvinna ska bonusen vara 20%, om en man -20%.

In [ ]:
def metoo(gender, salery):
    if gender == 'F':
        bonus = salery * 0.2
    else:
        bonus = salery * (-0.2)
    return bonus

In [ ]:
bonus = metoo('M',100)
bonus

### OBS! Förstår man nedan har man ett mycket kraftfullt verktyg för att manipupelera data på radnivå med Pandas

In [ ]:
df_staff['Mee_to_Bonus'] = df_staff.apply(lambda x: metoo(x['Gender'], x['Salary']), axis = 1) 

### Metoden apply appliceras på objektet df_staff. Denna metod tillåter att man skickar in en funktion. I exemplet ovan är det en lambda funktion som använder en egendefineradfunktion som argument (metoo). På varje rad appliceras funktionen metoo med argumenten df_staff['Gender'] och df_staff['Salary]. Lambda funktionen erbjuder möjligheter att använda funktioner i sammanhang där den vanliga funktionen inte fungerar. Det är väl använd tid att sätta sig in i lambda konceptet.

In [ ]:
df_staff[['Gender','Salary','Mee_to_Bonus']].head(10)

In [ ]:
?df_staff.apply

In [ ]:
type(df_staff.apply)

## 8. Joina dataframes 

In [ ]:
df_cust = pd.read_sas('customer2.sas7bdat', encoding = 'Latin8')
df_trans = pd.read_sas('transaction2.sas7bdat', encoding = 'Latin8')

In [ ]:
df_cust.info()

In [ ]:
df_trans.info()

In [ ]:
df_cust.head()

In [ ]:
df_cust = df_cust.drop_duplicates('ID')
df_cust.head()

In [ ]:
df_trans.head()

In [ ]:
# Inner join
df_merged_inner = df_cust.merge(df_trans, on = ['ID'], how = 'inner')
df_merged_inner.head()


In [ ]:
# Left join 

df_merged_left = df_cust.merge(df_trans, on = ['ID'], how = 'left')
df_merged_left.head()

In [ ]:
# Outer join

df_merged_outer = df_cust.merge(df_trans, on = ['ID'], how = 'outer')
df_merged_outer.head(6)

## 9. Konkatinera dataframes

In [ ]:
df_merged = df_merged_outer.copy() 

In [ ]:
df_merged

In [ ]:
concat_df = pd.concat([df_merged, df_merged_outer ], axis = 0)

In [ ]:
concat_df.head(20)

## 9. Split - Apply - Combine
### Aggregering - summera data i en serie för att returnera en skalär.  Vi vill ta fram statistik över en gruppering på data: split- apply-combine konceptet

* Split - Datat delas upp i delar på vald grupperingsvariabel
* Apply - På delarna appliceras logik som returnerar skalär
* Combine - Delarna sätts ihop på grupperingsvariabeln

In [ ]:
# Läser in nytt exempeldata.

df_order_fact = pd.read_sas('order_fact.sas7bdat', encoding = 'Latin8')
df_product_dim = pd.read_sas('product_dim.sas7bdat', encoding = 'Latin8')

In [ ]:
df_product_dim.info()

In [ ]:
df_product_dim.head()

In [ ]:
df_order_fact.info()

In [ ]:
### Joinar fakta tabell mot dimensonstabellen och lägger på produktkategori

df_analys = df_order_fact.merge(df_product_dim[['Product_ID','Product_Line','Product_Category']],
                                on = ['Product_ID'], how = 'inner')

In [ ]:
df_analys.info()

In [ ]:
df_analys['Product_Category'].value_counts()

In [ ]:
df_analys['Product_Line'].value_counts()

In [ ]:
# Andel av totalen
round(df_analys['Product_Line'].value_counts()/len(df_analys),2)

### Nu vill vi ta fram statistik på grupperingen Product Line - vi använder split - apply - combine konceptet. Först med Pandas optimerade metoder. Dessa anropas med agg metoden - returnerar skalär 

In [ ]:
grouped = df_analys.groupby(['Product_Line','Product_Category']) 

In [ ]:
result = grouped['Total_Retail_Price'].agg('mean')
result

In [ ]:
### Du kan använda flera funktioner i samma anrop
funtions = ['count','mean','max']
result1 = grouped['Total_Retail_Price'].agg(funtions)
result1



### Om vi inte är nöjda med de defaulta namnen -använd en tuple

In [ ]:
### Du kan använda flera funktioner i samma anrop
funtions2 = [('N','count'),('Medel','mean'),('Max','max')]

result2 = grouped['Total_Retail_Price'].agg(funtions2)
result2

### Slutligen gör vi en egen funktion som vi använder apply mot

### Vi vill plocka de fem högs betalda anställda per kön i df_staff

In [ ]:
df_staff.info()

In [ ]:
# Gör din egna funktion
def top(df, n = 5, column ='Salary'): 
    return df.sort_values(by = column)[-n:]

In [ ]:
top(df_staff)

In [ ]:
# Nu grupperar vi på kön och plockar de med högsta lönerna per grupp

df_staff.groupby('Gender').apply(top)

## 10. Visualisering - Matplotlib
### Det finns ett helt ekosystem för visaulisering. Grundmodulen för detta är matplotlib som redan är importerad i denna notebook. Vi ska göra två enkla visualiseringar för att visa konceptet

In [ ]:
# Definierar hela arean
fig = plt.figure(figsize = (10,5))
# Skapar grafobjekt som ska läggas in i arean ovan
ax = fig.add_subplot(1,1,1)

ax.set_title('Inkomst fördelat på kön')

bar_serie = round(df_staff.groupby('Gender')['Salary'].mean())

bar_serie.plot(kind='bar', rot = 0, grid = False, alpha = 0.6)

plt.show()

### Vad ska man tänka på här? Det finns oändligt många parametrar att sätta?
### Alla dataframes och serier har en plot metod. 
### Om man sätter ett index blir detta index alltid X axeln
### Det enda sättet är att pröva sig fram med olika grafer, se hjälpen nedan
### Bra att känna till är att man kan annotera samt skapa flera grafer i samma area


In [ ]:
?bar_serie.plot

### Exenmpel annotering samt två grafer i samma bild

In [ ]:
fig = plt.figure(figsize = (18,9))
# Skapar grafobjekt som ska läggas in i arean ovan
# Graf1
ax1 = fig.add_subplot(2,1,1)
# Graf2
ax2 = fig.add_subplot(2,1,2)

ax1.set_title('Inkomst fördelat på kön')

ax2.set_title('Antal anställda per år')

bar_serie = round(df_staff.groupby('Gender')['Salary'].mean())

line_serie = round(df_staff.groupby('Year')['Gender'].count())

bar_serie.plot(kind='bar', ax = ax1, rot = 0, grid = False, alpha = 0.6)

line_serie.plot(kind='line', ax = ax2, rot = 0, style = 'k-')

### Nedan logik för annotering i graf2

ax2.annotate('Vad händer 2010?', 
            xy = (2010, 95),
            xytext = (2000, 80),
            arrowprops = dict(facecolor = 'black', shrink = 0.1, width = 2),
            horizontalalignment = 'left')

plt.show()

In [ ]:
?ax2.annotate

In [ ]:
bar_serie

In [ ]:
type(bar_serie)

In [ ]:
line_serie

### Objekthantering när du skapar en ny dataframe, vy eller fysiskt data
__Det sätt som Pandas skapar en ny dataframe, vy eller nytt fysiskt data kan ge varningen "SettingWithCopyWarning". Jag kommer inte att ge me in på djupet på detta men nedan ett exempel. När denna varning uppstår så försöker ni göra något mot en dataframe via en vy som inte slår igenom i det underliggande datat. Och man ska också vara medveten om att i vissa fall så arbetar men mot en vy som faktiskt slår igenom i det underliggande datat__

### Först manipulation av vy som påverkar ett underliggande objekt

In [ ]:
df_order_fact = pd.read_sas('order_fact.sas7bdat', encoding = 'Latin8')

In [ ]:
id(df_order_fact)

In [ ]:
df_new = df_order_fact

In [ ]:
id(df_new)

In [ ]:
df_order_fact.info()

In [ ]:
df_new['Q2'] = df_new.Quantity * df_new.Quantity

In [ ]:
df_new.info()

In [ ]:
df_order_fact.info()

### Vill man vara helt säker på vad man gör använd copy metod för att skapa ett nytt objekt

In [ ]:
df_order_fact2 = pd.read_sas('order_fact.sas7bdat', encoding = 'Latin8')

In [ ]:
id(df_order_fact2)

In [ ]:
df_new2 = df_order_fact2.copy()

In [ ]:
id(df_new2)

In [ ]:
df_new2['Q2'] = df_new2.Quantity * df_new2.Quantity 

In [ ]:
df_new2.info()

In [ ]:
df_order_fact2.info()

__Nu provocerar vi fram en SettingWithWarningCopy__

In [ ]:
df_new

__Låt oss anta att för alla rader med Quantity = 1 är fel, de ska vara 10 istället. Vi måste uppdarera dataframen__

In [ ]:
df_new[df_new.Quantity == 1]['Q2'] = 100

In [ ]:
df_new[df_new.Quantity == 1][['Quantity','Q2']]

### Försök till förklaring

__Metoden df_new[df_new.Quantity == 1] skapar en vy av den ursprunliga dataframen (get method) , sedan gör vi en assignment ['Q2'] = 100 (assigmnent method) som inte slår igenom mot det underliggande objektet, det fysiska data. Lösningen är att arbete med loc metoden som resulterar i en operation och som säkerställer att det underliggande datat blir uppdaterat__ 

### Slicing loc metoden
* Välj rad och kolumn genom att ange index och variabel

In [ ]:
df_new.loc[df_new.Quantity == 1,'Q2'] = 100

In [ ]:
df_new[df_new.Quantity == 1][['Quantity','Q2']]

In [ ]:
id(df_new)

In [ ]:
id(df_order_fact)

In [ ]:
df_order_fact[df_order_fact.Quantity == 1][['Quantity','Q2']]

# Take away!

__Aldrig ignorerar denna varning!.__

# Slicing in a dataframe: Skär ut rader och kolumner genom positioner (index) - iloc

In [ ]:
# df_new är en vy mot df_order_fact
df_new

In [ ]:
df_new.iloc?

In [ ]:
## Välja kolumner utan metod genom att peka på kolumner
df_new[['Customer_ID','Employee_ID']]

In [ ]:
# Select row and columns with iloc method
df_new.iloc[0,0]

In [ ]:
df_new.iloc[1,0]

In [ ]:
df_new.iloc[:2,:2]

In [ ]:
df_new.iloc[-4:,:2]

In [ ]:
df_order_fact2.loc?